# New Section

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
#!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [4]:
#Data augmentation

image_gen = ImageDataGenerator(rotation_range=80, # rotate the image 30 degrees
                               width_shift_range=0.3, # Shift the pic width by a max of 30%
                               height_shift_range=0.3, # Shift the pic height by a max of 30%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=[0.2, 1.6], # Zoom in by 160% max, -20% min
                               horizontal_flip=True, # Allo horizontal flipping
                               vertical_flip=True,
                               fill_mode='wrap') # Fill in missing pixels with the nearest filled value                          

In [5]:
pwd

'C:\\python\\Data Science Training\\Prjects\\glucoma'

In [6]:
train_image_gen = image_gen.flow_from_directory('dataset/train',
                                               target_size=(192,192),
                                               batch_size=5,
                                               class_mode='binary')

Found 60 images belonging to 2 classes.


In [7]:
test_image_gen = image_gen.flow_from_directory('dataset/test',
                                               target_size=(192,192),
                                               batch_size=5,
                                               class_mode='binary')

Found 20 images belonging to 2 classes.


In [ ]:
def build_model(hp):
    model = keras.Sequential()

    model.add(Conv2D(filters=hp.Int("first_conv", min_value=16, max_value=128, step=16), 
                     kernel_size=(3, 3), input_shape=(192, 192, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # for i in range(hp.Int("n_layers", 1, 4)):
    #     model.add(Conv2D(hp.Int(f"second_conv{i}", min_value=8, max_value=128, step=8), (3, 3)))
    #     model.add(Activation('relu'))

    model.add(Flatten())  

    model.add(Dense(1))
    model.add(Activation("sigmoid"))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 3e-2, 3e-3, 3e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=["accuracy"])
    
    return model

In [ ]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=30,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(train_image_gen, 
             epochs=50, 
             validation_data=test_image_gen, 
             callbacks=[stop_early])

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get('first_conv'))
print(best_hps.get('learning_rate'))

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_dataset, epochs=50, validation_data=test_dataset)

In [ ]:
model.evaluate(train_dataset)

In [ ]:
model.evaluate(test_dataset)

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image

#C:\python\Data Science Training\Prjects\glucoma\non-glucoma\1 (13).jpg
#C:\python\Data Science Training\Prjects\glucoma\Glaucoma\2 (13).jpg

img_file = r'C:\python\Data Science Training\Prjects\glucoma\Glaucoma\2 (11).jpg'

img_file = image.load_img(img_file, target_size=(192, 192))

# img_file = image.img_to_array(img_file)
img_file = np.array(img_file)

img_file = np.expand_dims(img_file, axis=0)
img_file = img_file/255


prediction_prob = saved_model.predict(img_file)
print(prediction_prob)

[[0.00140059]]


In [11]:
model1 = keras.Sequential()

model1.add(Conv2D(128, kernel_size=(3, 3), input_shape=(192, 192, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Flatten())  

model1.add(Dense(1))
model1.add(Activation("sigmoid"))

model1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=["accuracy"])

In [12]:
history = model1.fit(train_image_gen, epochs=20, validation_data=test_image_gen)

Epoch 1/20
12/12 [==============================] - 5s 423ms/step - loss: 7.6429 - accuracy: 0.4667 - val_loss: 3.5417 - val_accuracy: 0.5000
Epoch 2/20
12/12 [==============================] - 5s 386ms/step - loss: 0.9727 - accuracy: 0.6667 - val_loss: 0.6511 - val_accuracy: 0.5500
Epoch 3/20
12/12 [==============================] - 4s 372ms/step - loss: 0.4875 - accuracy: 0.7333 - val_loss: 0.4434 - val_accuracy: 0.8500
Epoch 4/20
12/12 [==============================] - 5s 413ms/step - loss: 0.4173 - accuracy: 0.7667 - val_loss: 0.5788 - val_accuracy: 0.7000
Epoch 5/20
12/12 [==============================] - 5s 413ms/step - loss: 0.4143 - accuracy: 0.8333 - val_loss: 0.3844 - val_accuracy: 0.8500
Epoch 6/20
12/12 [==============================] - 4s 373ms/step - loss: 0.2387 - accuracy: 0.9333 - val_loss: 0.2743 - val_accuracy: 0.8500
Epoch 7/20
12/12 [==============================] - 4s 370ms/step - loss: 0.2279 - accuracy: 0.9000 - val_loss: 0.2055 - val_accuracy: 0.9000
Epoch 

In [13]:
model1.evaluate(train_image_gen)

12/12 [==============================] - 2s 198ms/step - loss: 0.1128 - accuracy: 0.9667


[0.11278032511472702, 0.9666666388511658]

In [14]:
model1.evaluate(test_image_gen)

4/4 [==============================] - 1s 149ms/step - loss: 0.0863 - accuracy: 0.9500


[0.08633328974246979, 0.949999988079071]

In [15]:
model1.save('models/model_1.h5')

In [2]:
saved_model = tf.keras.models.load_model('models/model_1.h5')

In [17]:
saved_model.evaluate(train_image_gen)

12/12 [==============================] - 2s 197ms/step - loss: 0.0934 - accuracy: 0.9667


[0.09341981261968613, 0.9666666388511658]

In [18]:
saved_model.evaluate(test_image_gen)

4/4 [==============================] - 1s 161ms/step - loss: 0.1041 - accuracy: 0.9500


[0.10405488312244415, 0.949999988079071]